In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('C:/Users/hongz/Downloads/choices13k-main/problems.csv')

# Assuming 'data' is the DataFrame containing the dataset
# Extracting the Y variable ('errors') and X variables (from 'State' to 'ProbDiffSqr')
Y = data['errors']
X = data.loc[:, 'State':'ProbDiffSqr']

In [ ]:
# Check for infinite values
if np.any(np.isinf(X)) or np.any(np.isnan(X)):
    print("Data contains infinite or NaN values")

In [3]:
# Replace infinite values with nan
X = np.where(np.isinf(X), np.nan, X)

# Option 1: Remove rows with NaN values
X_cleaned = X[~np.isnan(X).any(axis=1)]

# Option 2: Replace NaN values with the mean of each column
col_mean = np.nanmean(X, axis=0)
inds = np.where(np.isnan(X))
X[inds] = np.take(col_mean, inds[1])

if not np.any(np.isinf(X)) and not np.any(np.isnan(X)):
    print("Data is clean")

Data is clean


In [23]:
X_train.to_csv('kdjd.csv')

In [24]:
Y_train.to_csv('kdd.csv')

In [25]:
X_test.to_csv('kjd.csv')

In [26]:
Y_test.to_csv('kd.csv')

In [14]:
# Splitting the data into a train set (75%) and a test set (25%)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

# Using LassoCV to find the best alpha (penalty parameter) that minimizes the mean squared error
lasso = LassoCV(cv=5, random_state=42).fit(X_train, Y_train)

# Display the best alpha value found
print(f"Best alpha using LassoCV: {lasso.alpha_}")

# Coefficients of the model
print(f"Lasso coefficients: {lasso.coef_}")

# Predicting on the test set
Y_pred = lasso.predict(X_test)

# Calculating the mean squared error on the test set
mse = mean_squared_error(Y_test, Y_pred)
print(f"Mean Squared Error on Test Set: {mse}")

# Identifying non-zero coefficients (features) - ensuring X is treated as DataFrame
non_zero_indices = lasso.coef_ != 0  # This creates a boolean mask


Best alpha using LassoCV: 1040.2228252828427
Lasso coefficients: [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  6.09333185e-09  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  4.04883662e-09
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
Mean Squared Error on Test Set: 0.032281742358297255


In [22]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
non_zero_features = X_train.columns[non_zero_indices]  # Apply the mask to the columns of the training DataFrame
print(f"Features with non-zero coefficients: {list(non_zero_features)}")

Features with non-zero coefficients: [25, 31]


In [16]:
from sklearn.metrics import explained_variance_score

# Calculating the explained variance score
explained_variance = explained_variance_score(Y_test, Y_pred)

print(f"Explained Variance Score: {explained_variance}")


Explained Variance Score: 0.03611752861407891


In [29]:
lasso.score(X_train,Y_train)

0.026105723322867425

In [34]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
results = model.fit(X,Y)

beta_coeffs = model.coef_

print(results.summary())

AttributeError: 'LinearRegression' object has no attribute 'summary'

In [35]:
import statsmodels.api as sm

# Adding a constant term for the intercept
X_with_constant = sm.add_constant(X)

# Fitting the model
model = sm.OLS(Y, X_with_constant)
results = model.fit()

# Printing the summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                 errors   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     82.97
Date:                Thu, 04 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:45:06   Log-Likelihood:                 5878.2
No. Observations:               14568   AIC:                        -1.166e+04
Df Residuals:                   14519   BIC:                        -1.129e+04
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0018      0.013     -0.139      0.8